<a href="https://colab.research.google.com/github/OliverZhuUW/stand_mapping/blob/master/Multichannel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch.nn as nn
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN


In [31]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/Shared drives/stand_mapping/')
!pip install rasterio

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 18.2MB 63.9MB/s 


In [32]:
backbone = resnet_fpn_backbone(backbone_name='resnet50', pretrained=False)
backbone.body.conv1 = nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
backbone 
#model = resnet50_fpn()


model = MaskRCNN(backbone, 1)
model

#have 4 channel now, still need do actually, train 
#model. predict, apply the image. 
#get the model to predict something

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequentia

In [0]:
def open_raster(path_to_file):
  with rasterio.open(path_to_file) as src:
    raster = src.read()
    img = reshape_as_image(raster)
    return img 

In [35]:
import glob
IMG_DIR = './data/processed/packforest/training_chips_1m/'
TILE_ID = '552891_5186710_512'
glob.glob(os.path.join(IMG_DIR, TILE_ID + '*.tif'))

['./data/processed/packforest/training_chips_1m/552891_5186710_512_stand_mask.tif',
 './data/processed/packforest/training_chips_1m/552891_5186710_512_naip.tif',
 './data/processed/packforest/training_chips_1m/552891_5186710_512_cover_hard.tif',
 './data/processed/packforest/training_chips_1m/552891_5186710_512_cover_soft.tif',
 './data/processed/packforest/training_chips_1m/552891_5186710_512_biomass.tif']

In [0]:
from skimage.util import img_as_float
import rasterio
from rasterio.plot import reshape_as_image
import numpy as np

In [0]:
naip = img_as_float(open_raster(os.path.join(IMG_DIR, TILE_ID + '_naip.tif')))
gt_mask = open_raster(os.path.join(IMG_DIR, TILE_ID + '_stand_mask.tif'))
biomass = open_raster(os.path.join(IMG_DIR, TILE_ID + '_biomass.tif'))
biomass = biomass/biomass.max() # rescale biomass to have maximum of 1

In [48]:
test_img = np.dstack([naip, biomass])
test_img.shape

(512, 512, 4)

In [0]:
class RGBMaskEncoderCNN(nn.Module):
	def __init__(self):
		super(RGBMaskEncoderCNN, self).__init__()

		self.alexnet = models.alexnet(pretrained=True)
		# get the pre-trained weights of the first layer
		pretrained_weights = self.alexnet.features[0].weight
		new_features = nn.Sequential(*list(self.alexnet.features.children()))
        #here is 4 channels 
		new_features[0] = nn.Conv2d(4, 64, kernel_size=11, stride=4, padding=2)

		# For M-channel weight should randomly initialized with Gaussian
		new_feaures_[0].weight.data.normal_(0, 0.001)
		# For RGB it should be copied from pretrained weights
		new_features[0].weight.data[:, :3, :, :] = pretrained_weights

		self.alexnet.features = new_features

		
		

	def forward(self, images):
		"""Extract Feature Vector from Input Images"""
		features = self.alexnet(images)
		return features